# Stats Compass Core - Multi-DataFrame Operations

This notebook demonstrates stats-compass-core's ability to manage and combine multiple DataFrames simultaneously - a key differentiator for stateful data orchestration.

**Features demonstrated:**
- Managing multiple DataFrames in a single session
- SQL-style merges (inner, left, right, outer joins)
- Concatenating DataFrames (vertical and horizontal)
- Chaining operations across DataFrames

In [ ]:
# Setup
from stats_compass_core import DataFrameState, registry
import pandas as pd
import numpy as np

# Initialize state
state = DataFrameState()
registry.auto_discover()
print("Stats Compass Core ready!")

---
## Part 1: Creating Multiple DataFrames

Let's simulate a realistic scenario: an e-commerce database with customers, orders, and products.

In [ ]:
# Create Customers table
customers = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'city': ['London', 'Paris', 'Berlin', 'Madrid', 'Rome'],
    'signup_date': pd.to_datetime(['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-05-01'])
})
state.set_dataframe(customers, name='customers', operation='create')
print("Customers:")
customers

In [ ]:
# Create Orders table
orders = pd.DataFrame({
    'order_id': [101, 102, 103, 104, 105, 106, 107],
    'customer_id': [1, 2, 1, 3, 2, 6, 1],  # Note: customer 6 doesn't exist!
    'product_id': ['P001', 'P002', 'P001', 'P003', 'P001', 'P002', 'P003'],
    'quantity': [2, 1, 3, 1, 2, 1, 1],
    'order_date': pd.to_datetime(['2023-06-01', '2023-06-02', '2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07'])
})
state.set_dataframe(orders, name='orders', operation='create')
print("Orders:")
orders

In [ ]:
# Create Products table
products = pd.DataFrame({
    'product_id': ['P001', 'P002', 'P003', 'P004'],
    'product_name': ['Widget', 'Gadget', 'Gizmo', 'Doohickey'],
    'price': [29.99, 49.99, 19.99, 99.99],
    'category': ['Electronics', 'Electronics', 'Tools', 'Premium']
})
state.set_dataframe(products, name='products', operation='create')
print("Products:")
products

In [ ]:
# List all DataFrames in state
from stats_compass_core.data.list_dataframes import list_dataframes, ListDataFramesInput

result = list_dataframes(state, ListDataFramesInput())
print(f"DataFrames in state: {result.total_count}\n")
for df_info in result.dataframes:
    print(f"  - {df_info['name']}: {df_info['shape'][0]} rows × {df_info['shape'][1]} cols")

---
## Part 2: SQL-Style Merges

Use `merge_dataframes` to join tables like SQL JOINs.

In [ ]:
from stats_compass_core.data.merge_dataframes import merge_dataframes, MergeDataFramesInput

In [ ]:
# INNER JOIN: Orders with customer names (only matching records)
result = merge_dataframes(state, MergeDataFramesInput(
    left_dataframe='orders',
    right_dataframe='customers',
    how='inner',
    on='customer_id',
    save_as='orders_with_customers'
))

print(result.message)
print(f"\nNote: Order 106 (customer_id=6) was excluded - no matching customer!")
state.get_dataframe('orders_with_customers')

In [ ]:
# LEFT JOIN: All orders, with customer info where available
result = merge_dataframes(state, MergeDataFramesInput(
    left_dataframe='orders',
    right_dataframe='customers',
    how='left',
    on='customer_id',
    save_as='orders_left_join'
))

print(result.message)
print("\nNote: Order 106 is included, but customer info is NaN:")
state.get_dataframe('orders_left_join')

In [ ]:
# OUTER JOIN: All customers and all orders
result = merge_dataframes(state, MergeDataFramesInput(
    left_dataframe='customers',
    right_dataframe='orders',
    how='outer',
    on='customer_id',
    save_as='full_outer'
))

print(result.message)
print("\nCustomers 4 & 5 have no orders, customer 6 has no profile:")
state.get_dataframe('full_outer')

In [ ]:
# Chain merge: Add product details to orders
# First, merge orders with customers, then with products

# Step 1: Orders + Customers
merge_dataframes(state, MergeDataFramesInput(
    left_dataframe='orders',
    right_dataframe='customers',
    how='left',
    on='customer_id',
    save_as='temp_orders_customers'
))

# Step 2: (Orders + Customers) + Products
result = merge_dataframes(state, MergeDataFramesInput(
    left_dataframe='temp_orders_customers',
    right_dataframe='products',
    how='left',
    on='product_id',
    save_as='complete_orders'
))

print("Complete order details with customer and product info:")
df = state.get_dataframe('complete_orders')
df[['order_id', 'name', 'product_name', 'quantity', 'price']]

In [ ]:
# Calculate order totals
df = state.get_dataframe('complete_orders')
df['total'] = df['quantity'] * df['price']
state.set_dataframe(df, name='complete_orders', operation='calculate_totals')

print("Order totals:")
df[['order_id', 'name', 'product_name', 'quantity', 'price', 'total']]

---
## Part 3: Concatenating DataFrames

Use `concat_dataframes` to stack DataFrames vertically (add rows) or horizontally (add columns).

In [ ]:
from stats_compass_core.data.concat_dataframes import concat_dataframes, ConcatDataFramesInput

In [ ]:
# Scenario: Monthly sales data arriving in batches
jan_sales = pd.DataFrame({
    'date': pd.to_datetime(['2024-01-05', '2024-01-15', '2024-01-25']),
    'revenue': [1200, 1500, 1800],
    'region': ['North', 'South', 'East']
})
state.set_dataframe(jan_sales, name='jan_sales', operation='create')

feb_sales = pd.DataFrame({
    'date': pd.to_datetime(['2024-02-05', '2024-02-15', '2024-02-25']),
    'revenue': [1400, 1600, 2000],
    'region': ['North', 'South', 'East']
})
state.set_dataframe(feb_sales, name='feb_sales', operation='create')

mar_sales = pd.DataFrame({
    'date': pd.to_datetime(['2024-03-05', '2024-03-15', '2024-03-25']),
    'revenue': [1700, 1900, 2200],
    'region': ['North', 'South', 'East']
})
state.set_dataframe(mar_sales, name='mar_sales', operation='create')

print("Created 3 monthly sales DataFrames")

In [ ]:
# Vertical concatenation: Stack all months together
result = concat_dataframes(state, ConcatDataFramesInput(
    dataframes=['jan_sales', 'feb_sales', 'mar_sales'],
    axis=0,  # Stack rows
    save_as='q1_sales'
))

print(result.message)
print("\nQ1 Sales (all months combined):")
state.get_dataframe('q1_sales')

In [ ]:
# Horizontal concatenation: Combine different feature sets
demographics = pd.DataFrame({
    'age': [25, 35, 45, 55],
    'income': [50000, 75000, 90000, 120000]
})
state.set_dataframe(demographics, name='demographics', operation='create')

behavior = pd.DataFrame({
    'visits': [10, 25, 5, 15],
    'purchases': [2, 8, 1, 4]
})
state.set_dataframe(behavior, name='behavior', operation='create')

result = concat_dataframes(state, ConcatDataFramesInput(
    dataframes=['demographics', 'behavior'],
    axis=1,  # Add columns
    save_as='customer_features'
))

print(result.message)
print("\nCombined customer features:")
state.get_dataframe('customer_features')

In [ ]:
# Handling mismatched columns with outer join (default)
df_a = pd.DataFrame({'x': [1, 2], 'y': [3, 4]})
state.set_dataframe(df_a, name='df_a', operation='create')

df_b = pd.DataFrame({'x': [5, 6], 'z': [7, 8]})  # Has 'z' instead of 'y'
state.set_dataframe(df_b, name='df_b', operation='create')

# Outer join - keeps all columns
result = concat_dataframes(state, ConcatDataFramesInput(
    dataframes=['df_a', 'df_b'],
    axis=0,
    join='outer',
    save_as='outer_concat'
))

print("Outer concat (keeps all columns, fills NaN):")
state.get_dataframe('outer_concat')

In [ ]:
# Inner join - keeps only common columns
result = concat_dataframes(state, ConcatDataFramesInput(
    dataframes=['df_a', 'df_b'],
    axis=0,
    join='inner',
    save_as='inner_concat'
))

print("Inner concat (keeps only common columns):")
state.get_dataframe('inner_concat')

---
## Part 4: Real-World Workflow

Combine merge and concat in a realistic data pipeline.

In [ ]:
# Scenario: Analyze Q1 sales by product category

# First, let's use our complete_orders DataFrame
orders_df = state.get_dataframe('complete_orders')
print(f"Working with {len(orders_df)} complete orders")

# Group by category and calculate totals using transforms
from stats_compass_core.transforms.groupby_aggregate import groupby_aggregate, GroupByAggregateInput

result = groupby_aggregate(state, GroupByAggregateInput(
    dataframe_name='complete_orders',
    by=['category'],
    aggregations={'total': ['sum', 'mean', 'count']},
    save_as='sales_by_category'
))

print("\nSales by Category:")
state.get_dataframe('sales_by_category')

In [ ]:
# Final state summary
summary = state.get_state_summary()

print("=" * 60)
print("SESSION SUMMARY")
print("=" * 60)
print(f"\nTotal DataFrames managed: {summary['dataframes']['count']}")
print(f"Memory used: {summary['memory']['used_mb']:.2f} MB")
print(f"\nDataFrames created:")
for name in summary['dataframes']['names']:
    df = state.get_dataframe(name)
    print(f"  - {name}: {df.shape[0]} rows × {df.shape[1]} cols")

---
## Summary

Stats Compass Core's stateful architecture enables:

1. **Multi-DataFrame Management** - Hold multiple datasets in memory with named references
2. **SQL-Style Merges** - Join tables with inner/left/right/outer joins
3. **Flexible Concatenation** - Stack data vertically or horizontally
4. **Chainable Operations** - Build complex pipelines by referencing previous results

This makes it ideal for:
- Data integration workflows
- Multi-source analytics
- Building AI-powered data agents that need to orchestrate across datasets